In [14]:
import pandas as pd

In [15]:
import math

In [16]:
from scipy import stats as sc

In [17]:
from matplotlib import pyplot as plt

In [18]:
import numpy as np

In [4]:
from scipy import stats

In [5]:
csv = pd.read_csv('./ocean157ksafe.csv', ',')
print(len(csv.index))
# csv = pd.read_csv('../1605298557-experimento-personalidades.csv', ';')
# print(len(csv.index))

157101


In [6]:
renames = {
    '1_extraversion': "Extroversión",
    '1_agreeableness': "Agradabilidad",
    '1_conscientiousness': "Responsabilidad",
    '1_neuroticism': "Neuroticismo",
    '1_openness': "Apertura a la experiencia",
}

percentiles = {
    "min": 0.25,
    "med": 0.5,
    "max": 0.75
}

traits = list(renames.values())

In [7]:
csv = csv.rename(columns=renames)

In [8]:
step = 1
inf = 18
sup = 60
aggr = csv.groupby(pd.cut(csv["2_edad"], np.arange(inf - 1, sup + step, step)))

In [9]:
data = {}
for group, values in aggr:
    edad = str(group)[-3:-1]
    data[edad] = {
        "min": [],
        "med": [],
        "max": []
    }
    for p in percentiles.keys():
        q = values.quantile(percentiles[p])
        for trait in traits:
            data[edad][p].append(q[trait])

In [10]:
def getEstimate(own):
    min_diff = {"k": 0, "diff": 10000}
    for item in list(data.items()):
        diff = 0
        key, values = item
        for i, value in enumerate(own):
            trait_diff = ((value - values['med'][i]) / (values['max'][i] - values['min'][i])) ** 2
            diff += trait_diff
        if diff < min_diff["diff"]:
            min_diff = {"k": key, "diff": diff}
    return min_diff

In [11]:
getEstimate([21, 32, 27, 29, 45])

{'k': '18', 'diff': 0.7827538422776518}

In [19]:
edades = np.array([])
estimates = np.array([])
diffs = np.array([])
for i, row in csv.iterrows():
    own = np.array([])
    for col in traits:
        own = np.append(own, row[col])
    estimate = getEstimate(list(own))
    estimates = np.append(estimates, int(estimate["k"]))
    edades = np.append(edades, int(row['2_edad']))
    diffs = np.append(diffs, (abs(int(row['2_edad']) - int(estimate["k"]))))

In [20]:
sc.pearsonr(list(edades), list(estimates))

(0.16492277516555565, 0.0)

In [21]:
coef = np.polyfit(edades, estimates, 1)

In [22]:
poly1d_fn = np.poly1d(coef)

In [ ]:
plt.plot(edades, estimates, 'yo', edades, poly1d_fn(edades), '--k')
plt.xlim(18,60)
plt.ylim(18,60)

(18.0, 60.0)